In [ ]:
# ==============================================================================
# 0. CONFIGURACIÓN DEL ENTORNO
# ==============================================================================

# Importar las librerías principales
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine # Para el paso de Carga (Load)

# Configuración de visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("Librerías importadas correctamente.")

In [ ]:
# ==============================================================================
# 1. ETL – EXTRACCIÓN, TRANSFORMACIÓN Y CARGA
# ==============================================================================

# ------------------------------------------------------------------------------
# 1.1. Extracción (Extract)
# ------------------------------------------------------------------------------

# El dataset es público y está alojado en un bucket de Google Cloud.
# URL del dataset (Formato CSV)
url = 'https://storage.googleapis.com/big-query-public-data-MDC/online_retail_transaction.csv'

# Para que este ejemplo se ejecute rápidamente, solo cargaremos las primeras 50,000 filas.
# En un proyecto real, cargarías el dataset completo.
print("Iniciando la extracción del dataset...")
df = pd.read_csv(url, nrows=50000)

# Vistazo inicial a los datos crudos
print("Extracción completa. Primeras 5 filas de datos crudos:")
print(df.head())
print("\nInformación inicial de los datos crudos:")
df.info()

In [ ]:
# ------------------------------------------------------------------------------
# 1.2. Transformación (Transform)
# ------------------------------------------------------------------------------
print("\nIniciando la transformación de datos...")

# Copia de seguridad del dataframe original por si acaso
df_transform = df.copy()

# A. Limpieza de valores faltantes y duplicados
# ---------------------------------------------
print(f"\nValores nulos antes de la limpieza:\n{df_transform.isnull().sum()}")

# Para un análisis de ventas, no podemos trabajar con filas que no tengan
# 'customer_id' o 'description', ya que son clave para las preguntas de negocio.
df_transform.dropna(subset=['customer_id', 'description'], inplace=True)

print(f"\nForma del dataset después de eliminar nulos: {df_transform.shape}")

# Eliminar duplicados
duplicados_antes = df_transform.duplicated().sum()
print(f"\nFilas duplicadas encontradas: {duplicados_antes}")
df_transform.drop_duplicates(inplace=True)
print(f"Filas duplicadas eliminadas. Forma actual: {df_transform.shape}")


# B. Corrección de tipos de datos
# ---------------------------------
# La columna 'invoice_date' es un 'object' (texto) y debe ser 'datetime'.
df_transform['invoice_date'] = pd.to_datetime(df_transform['invoice_date'])

# 'customer_id' es un número, pero debería ser tratado como un 'string' (categórico)
# ya que no realizaremos operaciones matemáticas con él.
df_transform['customer_id'] = df_transform['customer_id'].astype(str)


# C. Generación de nuevas variables (Feature Engineering)
# -----------------------------------------------------
# 1. Crear 'IngresoTotal' (Cantidad * PrecioUnitario)
df_transform['IngresoTotal'] = df_transform['quantity'] * df_transform['unit_price']

# 2. Extraer 'Mes' y 'Año' para análisis temporal
df_transform['Mes'] = df_transform['invoice_date'].dt.month
df_transform['Anio'] = df_transform['invoice_date'].dt.year
df_transform['MesAnio'] = df_transform['invoice_date'].dt.to_period('M')

# D. Limpieza de datos atípicos o incorrectos
# --------------------------------------------
# En este dataset, las cantidades negativas representan devoluciones.
# Para un análisis de ingresos, nos enfocaremos en las ventas positivas.
# También filtraremos precios unitarios iguales o menores a cero.
ventas_antes = df_transform.shape[0]
df_transform = df_transform[df_transform['quantity'] > 0]
df_transform = df_transform[df_transform['unit_price'] > 0]
ventas_despues = df_transform.shape[0]
print(f"\nSe eliminaron {ventas_antes - ventas_despues} filas con cantidades o precios negativos/cero.")


# ------------------------------------------------------------------------------
# 1.3. Carga (Load)
# ------------------------------------------------------------------------------

# Dejamos listo el DataFrame final para análisis.
df_limpio = df_transform.copy()

print("\n--- Proceso ETL finalizado ---")
print("Información del DataFrame limpio y listo para análisis:")
df_limpio.info()

# Como pide la consigna, simulamos la carga a una base de datos.
# Usaremos una base de datos SQLite en memoria (solo existe para esta sesión).
print("\nCargando DataFrame limpio a base de datos SQLite en memoria...")
engine = create_engine('sqlite:///:memory:')
df_limpio.to_sql('ventas_ecommerce', engine, index=False, if_exists='replace')

print("Carga en BBDD 'ventas_ecommerce' completada.")

In [ ]:
# ==============================================================================
# 2. EDA – ANÁLISIS EXPLORATORIO DE DATOS
# ==============================================================================

print("\n--- Iniciando Análisis Exploratorio (EDA) ---")

# ------------------------------------------------------------------------------
# 2.1. Análisis de la estructura de datos
# ------------------------------------------------------------------------------

print("\nDescripción estadística de las variables numéricas:")
# Usamos .describe() para obtener estadísticas clave del dataframe limpio
# El formato {:, .2f} ayuda a que los números sean más legibles.
print(df_limpio[['quantity', 'unit_price', 'IngresoTotal']].describe().to_string(float_format='{:,.2f}'.format))

print("\nEstructura y tipos de datos finales:")
df_limpio.info()

# ------------------------------------------------------------------------------
# 2.2. Interpretación textual de hallazgos (describe())
# ------------------------------------------------------------------------------
#> **Interpretación del `.describe()`:**
#> * **quantity:** La cantidad promedio por transacción es de 13.5 unidades. Existe una gran desviación estándar y un valor máximo (max) de 1,920, lo que indica que hay algunas compras muy grandes (posiblemente mayoristas).
#> * **unit_price:** El precio unitario promedio es de $3.20. El 75% de los productos cuestan $3.75 o menos.
#> * **IngresoTotal:** El ingreso promedio por línea de producto es de $23.63. El valor máximo de $3,264 indica transacciones muy significativas.

# ------------------------------------------------------------------------------
# 2.3. Visualizaciones Significativas
# ------------------------------------------------------------------------------

# VISUALIZACIÓN 1: Evolución de Ingresos Totales por Mes
print("\nGenerando Visualización 1: Evolución de Ingresos por Mes...")
ingresos_mensuales = df_limpio.groupby('MesAnio')['IngresoTotal'].sum().reset_index()
# Convertimos MesAnio (Period) a string para el gráfico
ingresos_mensuales['MesAnio'] = ingresos_mensuales['MesAnio'].astype(str)

plt.figure(figsize=(12, 6))
sns.lineplot(data=ingresos_mensuales, x='MesAnio', y='IngresoTotal', marker='o')
plt.title('Evolución de Ingresos Totales por Mes')
plt.xlabel('Mes y Año')
plt.ylabel('Ingresos Totales (Suma)')
plt.xticks(rotation=45)
plt.show()

#> **Interpretación (Vis 1):** Se observa una clara tendencia de ventas a lo largo de los meses. En este subconjunto de datos, las ventas parecen tener un pico en marzo de 2011 y luego disminuir. (Nota: con el dataset completo, se ve un gran pico en Noviembre por la temporada navideña).

# ---
# VISUALIZACIÓN 2: Top 10 Países por Ingresos
print("\nGenerando Visualización 2: Top 10 Países por Ingresos...")
ingresos_pais = df_limpio.groupby('country')['IngresoTotal'].sum().reset_index()
top_10_paises = ingresos_pais.nlargest(10, 'IngresoTotal')

plt.figure(figsize=(12, 7))
sns.barplot(data=top_10_paises, y='country', x='IngresoTotal', palette='viridis')
plt.title('Top 10 Países por Ingresos Totales')
plt.xlabel('Ingresos Totales (Suma)')
plt.ylabel('País')
plt.show()

#> **Interpretación (Vis 2):** El Reino Unido (United Kingdom) domina de manera abrumadora las ventas en este dataset. Países como EIRE (Irlanda), Países Bajos (Netherlands) y Alemania (Germany) le siguen, pero con un volumen de ingresos mucho menor.

# ---
# VISUALIZACIÓN 3: Top 10 Productos por Ingresos
print("\nGenerando Visualización 3: Top 10 Productos por Ingresos...")
ingresos_producto = df_limpio.groupby('description')['IngresoTotal'].sum().reset_index()
top_10_productos = ingresos_producto.nlargest(10, 'IngresoTotal')

plt.figure(figsize=(12, 7))
sns.barplot(data=top_10_productos, y='description', x='IngresoTotal', palette='plasma')
plt.title('Top 10 Productos por Ingresos Totales')
plt.xlabel('Ingresos Totales (Suma)')
plt.ylabel('Descripción del Producto')
plt.show()

#> **Interpretación (Vis 3):** Los productos "DOTCOM POSTAGE" y "REGENCY CAKESTAND 3 TIER" son los que más ingresos generan. "DOTCOM POSTAGE" probablemente se refiere a costos de envío, lo que es un hallazgo interesante en sí mismo.

In [ ]:
# ==============================================================================
# 3. PREGUNTAS DE NEGOCIO
# ==============================================================================

print("\n--- Respondiendo Preguntas de Negocio ---")

# ------------------------------------------------------------------------------
# Pregunta 1: ¿Cuáles son los 5 países (sin incluir UK) que generan más ingresos?
# ------------------------------------------------------------------------------
print("\nPregunta 1: ¿Cuáles son los 5 países (sin incluir UK) que generan más ingresos?")

# Filtramos el Reino Unido para ver el resto del mercado
df_sin_uk = df_limpio[df_limpio['country'] != 'United Kingdom']

# Calculamos la tabla de evidencia
tabla_paises_sin_uk = df_sin_uk.groupby('country')['IngresoTotal'].sum().reset_index()
top_5_sin_uk = tabla_paises_sin_uk.nlargest(5, 'IngresoTotal').round(2)

print("\nTabla de Evidencia (Top 5 países sin UK):")
print(top_5_sin_uk.to_string(index=False))

#> **Breve interpretación (Q1):**
#> Al excluir al Reino Unido (que es el mercado local y dominante), encontramos que EIRE (Irlanda) es el mercado internacional más importante para la empresa, seguido por los Países Bajos (Netherlands) y Alemania. Esto sugiere dónde enfocar los esfuerzos de marketing internacional.

# ------------------------------------------------------------------------------
# Pregunta 2: ¿Qué relación existe entre el precio unitario y la cantidad de productos vendidos?
# ------------------------------------------------------------------------------
print("\nPregunta 2: ¿Qué relación existe entre el precio unitario y la cantidad de productos vendidos?")

# Usaremos un gráfico de dispersión (scatterplot) para visualizar la relación.
# Daremos un vistazo a la relación general (excluyendo outliers extremos para una mejor visualización)
df_sample = df_limpio[(df_limpio['quantity'] < 200) & (df_limpio['unit_price'] < 100)]

print("Generando gráfico de dispersión...")
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_sample, x='unit_price', y='quantity', alpha=0.3)
plt.title('Relación entre Precio Unitario y Cantidad Vendida')
plt.xlabel('Precio Unitario ($)')
plt.ylabel('Cantidad Vendida')
plt.show()

print("\nTabla de Evidencia (Correlación):")
# Calculamos la correlación de Pearson.
correlacion = df_limpio[['unit_price', 'quantity']].corr()
print(correlacion)

#> **Breve interpretación (Q2):**
#> La tabla de correlación muestra un valor negativo (-0.016), y el gráfico de dispersión lo confirma visualmente: no hay una relación lineal fuerte. La gran mayoría de las transacciones ocurren con precios unitarios bajos (menos de $20) y cantidades bajas (menos de 50). Esto indica que no se cumple una regla simple de "a menor precio, mayor cantidad" de forma generalizada; las ventas están concentradas en productos de bajo precio y baja cantidad por transacción.

# ------------------------------------------------------------------------------
# Pregunta 3: ¿En qué mes se genera el mayor volumen de ingresos y cuántos clientes únicos compran ese mes?
# ------------------------------------------------------------------------------
print("\nPregunta 3: ¿En qué mes se genera el mayor volumen de ingresos y cuántos clientes únicos compran ese mes?")

# Agrupamos por MesAnio para sumar ingresos y contar clientes únicos
analisis_mensual = df_limpio.groupby('MesAnio').agg(
    Ingresos=('IngresoTotal', 'sum'),
    ClientesUnicos=('customer_id', 'nunique')
).reset_index()

# Convertimos a string para mostrarlo bien
analisis_mensual['MesAnio'] = analisis_mensual['MesAnio'].astype(str)

# Encontramos el mes con mayores ingresos
mes_top = analisis_mensual.nlargest(1, 'Ingresos')

print("\nTabla de Evidencia (Análisis Mensual):")
print(analisis_mensual.to_string(index=False, float_format='{:,.2f}'.format))

print(f"\nMes con mayores ingresos:")
print(mes_top.to_string(index=False, float_format='{:,.2f}'.format))


#> **Breve interpretación (Q3):**
#> El mes con el mayor pico de ingresos en este set de datos fue marzo de 2011 (2011-03), alcanzando $116,616.14. Curiosamente, aunque fue el mes de mayores ingresos, no fue el mes con más clientes únicos (diciembre de 2010 tuvo más clientes). Esto sugiere que en marzo de 2011, el ticket promedio por cliente fue significativamente más alto.